### Import the Data

In [2]:
import pandas as pd
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

# train_df = pd.read_csv('test_data/adults_train-test.csv')
train_df = pd.read_parquet('datasets/adults_train.parquet')

control_df = pd.read_csv('test_data/adults_control-test.csv')
synth_df = pd.read_parquet('test_data/adults_syn_ctgan.parquet')
synth_filepath = 'test_data/dataset_adults_train_ctgan_synthesizer.pkl'
synthesizer = CTGANSynthesizer.load(filepath=synth_filepath)

##make sure these libraries work
import pykeops 
#make sure omp.h is also working, may need to google if warning occurs or run this code block twice
#make sure all libraries are include, you may need to find the header files and move them locally so they can be imported
pykeops.clean_pykeops()   
pykeops.test_numpy_bindings() 
pykeops.test_torch_bindings() 


[KeOps] /Users/chhduong/.cache/keops2.1.2/Darwin_CHHDUONG-M-F2GZ_23.2.0_p3.9.6 has been cleaned.
[KeOps] Generating code for formula Sum_Reduction((Var(0,3,0)-Var(1,3,1))|(Var(0,3,0)-Var(1,3,1)),1) ... OK
[pyKeOps] Compiling pykeops cpp cb73cd1bce module ... OK
pyKeOps with numpy bindings is working!
pyKeOps with torch bindings is working!


### LabelOnlyDecisionBoundary?

In [3]:
# from art.attacks.inference.membership_inference import LabelOnlyDecisionBoundary


### DOMIAS Overfitting

In [3]:
#convert columns to numbers
# dummies = pd.get_dummies(
#     data=train_df, 
#     columns =['type_employer', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'country', 'income'], 
#     dtype=int
# )
# merged = pd.concat([train_df, dummies], axis='columns')
# merged.drop(
#     ['type_employer', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'country', 'income'], 
#     axis='columns', 
#     inplace=True
# )
# train_df = dummies
# train_df.to_parquet('test_data/temp_train.parquet')
# train_df = pd.read_parquet('test_data/temp_train.parquet')

## Easier to just drop all non-numerical columns
# train_df.drop(
#     ['type_employer', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'country', 'income'], 
#     axis='columns', 
#     inplace=True
# )

#smarter way to convert columns to numbers
transformed_df = train_df
for col in ['type_employer', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'country', 'income']:
    uniques = transformed_df[col].unique()
    for index, value in enumerate(uniques):
        transformed_df.loc[transformed_df[col] == value, col] = index+1

train_df = transformed_df
train_df.to_parquet('test_data/temp_train.parquet')
train_df = pd.read_parquet('test_data/temp_train.parquet')

## For some reason, country is causing the evaluator to run into an singular matrix exception.
train_df.drop(
    ['country'], 
    axis='columns', 
    inplace=True
)

/Users/chhduong/Library/Python/3.9/lib/python/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [5]:
from domias.evaluator import evaluate_performance
from domias.models.generator import GeneratorInterface
from sklearn.preprocessing import StandardScaler
import numpy as np

# DOMIAS Requires a generator derived from GeneratorInterface 
# The generator only requires 'fit' and 'generator' methods
# For testing, we set this up with CTGANSynthesizer
def get_generator(
    synthesizer,
    data
) -> GeneratorInterface:
    class LocalGenerator(GeneratorInterface):
        def __init__(self) -> None:
            self.model = synthesizer
            self.data = data
        def fit(self, data: pd.DataFrame) -> "LocalGenerator":
            self.model.fit(self.data)
            # self.model.fit(data)
            return self
        def generate(self, count: int) -> pd.DataFrame:
            return self.model.sample(count)
    return LocalGenerator()

mem_set_size = 30000
reference_set_size = 39000
training_epochs = 10
synthetic_sizes = [10000]
density_estimator = "prior"  # prior, kde, bnaf

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_df)
# metadata_dict = metadata.to_dict()
# print(metadata_dict)
dataset = train_df.to_numpy()
# scaler = StandardScaler()
# dataset = scaler.fit_transform(train_df)

synthesizer = CTGANSynthesizer(
    metadata=metadata,
    epochs=training_epochs,
)
generator = get_generator(synthesizer, train_df)

perf = evaluate_performance(
    generator,
    dataset,
    mem_set_size,
    reference_set_size,
    training_epochs=training_epochs,
    synthetic_sizes=synthetic_sizes,
    density_estimator=density_estimator,
)

/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will 

[KeOps] Generating code for formula Max_SumShiftExpWeight_Reduction(Concat(Var(2,1,1)-Var(3,1,2)*(((Var(0,1,0)-Var(1,1,1))*Sum(Var(0,1,0)-Var(1,1,1)))/2),1),0) ... OK
[pyKeOps] Compiling pykeops cpp c0c0296eaf module ... 

/Users/chhduong/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_

OK


/Users/chhduong/Library/Python/3.9/lib/python/site-packages/domias/baselines.py:38: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/Users/chhduong/Library/Python/3.9/lib/python/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your st

In [6]:
#TODO: Go through https://github.com/vanderschaarlab/DOMIAS/blob/main/tutorials/experiment_1_housing.ipynb

# print(perf)
# print(type(perf)

for key in perf[10000]:
    print(key)
    print(perf[10000][key])

# print(perf[10000]['data']['Xtest'][0])
# print(perf[10000]['data']['Ytest'][0])

# AUC by number of iterations
# output = pd.DataFrame([], columns=["epoch", "src", "aucroc"])
# for key in perf[10000]['MIA_performance']:
#     print("====" + str(key))
#     print(perf[10000]['MIA_performance'][key])
    # output = pd.concat(
    #     [
    #         output,
    #         pd.DataFrame(
    #             [
    #                 [training_epochs, key, perf[10000]['MIA_performance'][key]["aucroc"]],
    #             ],
    #             columns=["epoch", "src", "aucroc"],
    #         ),
    #     ]
    # )

MIA_performance
{'ablated_eq1': {'accuracy': 0.4954908792785407, 'aucroc': 0.4949393600531444}, 'ablated_eq2': {'accuracy': 0.4968231194916991, 'aucroc': 0.4953225642161204}, 'LOGAN_D1': {'accuracy': 0.5, 'aucroc': 0.4996700232506643}, 'MC': {'accuracy': 0.501024800163968, 'aucroc': 0.5013849959403602}, 'gan_leaks': {'accuracy': 0.39616212338593976, 'aucroc': 0.4979407624741659}, 'gan_leaks_cal': {'accuracy': 0.3880405820864931, 'aucroc': 0.4980730513728963}, 'LOGAN_0': {'accuracy': 0.49887271981963516, 'aucroc': 0.4964305118836729}, 'eq1': {'accuracy': 0.4954908792785407, 'aucroc': 0.4949393600531444}, 'domias': {'accuracy': 0.4954908792785407, 'aucroc': 0.4949393600531444}}
MIA_scores
{'ablated_eq1': array([2.18439302e-23, 3.83772329e-22, 1.88270983e-21, ...,
       1.32422853e-22, 1.73063919e-21, 1.04738489e-22]), 'ablated_eq2': array([0.00201283, 0.01096947, 0.01597137, ..., 0.00938871, 0.03192629,
       0.00599109]), 'LOGAN_D1': array([31771.479, 79236.36 , 55819.85 , ..., 56986.

### Save and Load Results

In [19]:
# results = perf[10000]['data']
# xtest = results['Xtest']
# ytest = results['Ytest']
# type(xtest)
# xtest_df = pd.DataFrame(xtest)
# train_ndarray = train_df.to_numpy()

# #result: xtest and train data are the same, ytest is just whether the train data row exists in the synthetic dataset
# anonymeter uses: n_attacks and n_success

mia_scores = perf[10000]['MIA_scores']['domias']
y_pred = mia_scores > np.median(mia_scores)
n_attacks = y_pred.size
n_success = (y_pred == True).sum()
print(n_attacks)
print(n_success)

39032
19516
